# Setting up required packages and code from old repo

In [3]:
!git clone https://github.com/kkahloots/Generative_Models.git # this is for loading git with correct brach
!cd  ./Generative_Models
!git checkout lmdb
!cd ..

Cloning into 'Generative_Models'...
remote: Enumerating objects: 2792, done.
remote: Counting objects: 100% (2792/2792), done.
remote: Compressing objects: 100% (782/782), done.
remote: Total 15616 (delta 2024), reused 2584 (delta 1853), pack-reused 12824
Receiving objects: 100% (15616/15616), 414.85 MiB | 16.89 MiB/s, done.
Resolving deltas: 100% (10625/10625), done.
Checking out files: 100% (1675/1675), done.
error: pathspec 'lmdb' did not match any file(s) known to git.


In [5]:
import os
os.chdir('/home/azeghost/git/LMDB_Datasets') # adress to git dir
print(os.getcwd())
from training.generators.file_image_generator import create_image_lists, get_generators
import numpy as np
import lmdb
import pickle
import sys
from keras.preprocessing.image import Iterator, load_img, img_to_array, array_to_img
from utils.data_and_files.file_utils import get_file_path

/home/azeghost/git/Generative_Models


# Store images in lmdb

In [6]:
images_dir = '/home/azeghost/git/Generative_Models/data/.pokemon/' #Folder to images
validation_percentage = 30
valid_format = 'png'

In [7]:
image_lists = create_image_lists(
    image_dir=images_dir, 
    validation_pct=validation_percentage, 
    valid_imgae_formats=valid_format,
    verbose = 1
)

  DEBUG    | Looking for images in 'DS06'
  INFO     | 809 file found


In [8]:
import os
os.chdir('/home/azeghost/git/LMDB_Datasets')

In [9]:
#Wrapper class for dataset
class Pokemon_Image:
    def __init__(self, image, label):
        self.channels = image.shape[2]
        self.size = image.shape[:2]
        self.image = image.tobytes()
        self.label = label #additional data to be stored (make it string)

    def get_image(self):
        """ Returns the image as a numpy array. """
        images = np.frombuffer(self.image, dtype=np.float32) #pay attention if you  don't use create_image_lists
        return images.reshape(*self.size, self.channels)     #then dtype will be different

In [10]:
def store_single_lmdb(filename, img , index, label, num_images):
    """ Stores a wrapper to LMDB.
    """
    map_size = num_images * img.nbytes * 10
    env = lmdb.open(filename, map_size=map_size)

    # Same as before — but let's write all the images in a single transaction
    with env.begin(write=True) as txn:
          # All key-value pairs need to be Strings
          value = Pokemon_Image(img, label)
          key = f"{index:08}"
          txn.put(key.encode("ascii"), pickle.dumps(value))

    env.close()

In [11]:
def store_lmdb(image_lists, image_dir, 
               lmdb_dir = '/content/LMDB',category='training', target_size=None,
               color_mode='rgb', save_prefix='', save_format='png'):
  classes = list(image_lists.keys())
  num_class = len(classes)
  class2id = dict(zip(classes, range(len(classes))))
  id2class = dict((v, k) for k, v in class2id.items())
 
  y = None
  X = None

  for label_name in classes:
    num_images = len(image_lists[label_name][category])
    print('Storing '+ str(num_images)+lmdb_dir+os.sep+'_{}'.format(category))
    for index, _ in enumerate(image_lists[label_name][category]):
        img_path = get_file_path(image_lists,
                                  label_name,
                                  index,
                                  image_dir,
                                  category)
        img = img_to_array(
                        load_img(
                        img_path,
                        grayscale=color_mode=='grayscale',
                        target_size=target_size
                        )
                    )
        name, _ = os.path.splitext(img_path)
        vid_img_arr = name.split(sep=os.sep)[-1:]
        y = [np.array(vid_img_arr)]     
        name =  lmdb_dir+os.sep+'_{}'.format(category)
        
        store_single_lmdb(index = index, filename=name, img = img,label = y ,num_images = num_images)

In [12]:
!rm -rf ./LMDB/
!mkdir ./LMDB/
# save_to_dir=None,
store_lmdb(image_lists, image_dir=images_dir, lmdb_dir = './LMDB'
           ,category='training',target_size=None,color_mode='rgb',save_prefix='',save_format='png')
store_lmdb(image_lists, image_dir=images_dir, lmdb_dir = './LMDB'
           ,category='validation',target_size=None,color_mode='rgb',save_prefix='',save_format='png')

Storing 591./LMDB/_training
Storing 218./LMDB/_validation


# Done on ubuntu system is local machine does not have tar 
 - Install git bash then u can use such commands.
 - Go to windows store and download Ubuntu emulator wich just installs bash
 - On colab these should work

 ## For Training

In [13]:
!tar -cvjf pokemon_training.tar.bz2 /home/azeghost/git/Generative_Models/LMDB/_training/* 

tar: Removing leading `/' from member names
/home/azeghost/git/Generative_Models/LMDB/_training/data.mdb
/home/azeghost/git/Generative_Models/LMDB/_training/lock.mdb


In [14]:
!split -b 10M pokemon_training.tar.bz2 "pokemon_training.tar.part"
#split -b <max size> <name of zip or dir to zip/name> <split file name beginning>

In [15]:
#Check if they are created
!ls -lh pokemon_training.tar.part*

-rw-r--r-- 1 azeghost azeghost  10M Jul 13 21:43 pokemon_training.tar.partaa
-rw-r--r-- 1 azeghost azeghost  10M Jul 13 21:43 pokemon_training.tar.partab
-rw-r--r-- 1 azeghost azeghost  10M Jul 13 21:43 pokemon_training.tar.partac
-rw-r--r-- 1 azeghost azeghost  10M Jul 13 21:43 pokemon_training.tar.partad
-rw-r--r-- 1 azeghost azeghost 4.4M Jul 13 21:43 pokemon_training.tar.partae


In [16]:
#move to correct folder and push to git
!mkdir /home/azeghost/git/LMDB_Datasets/pokemon
!mv pokemon_training.tar.part* /home/azeghost/git/LMDB_Datasets/pokemon
!ls -la /home/azeghost/git/LMDB_Datasets/pokemon 

mkdir: cannot create directory ‘/home/azeghost/git/LMDB_Datasets/pokemon’: File exists
total 181820
drwxr-xr-x 3 azeghost azeghost     4096 Jul 13 21:43 .
drwxr-xr-x 9 azeghost azeghost     4096 Jul 13 21:43 ..
drwxr-xr-x 3 azeghost azeghost     4096 Jul  1 22:44 home
-rw-r--r-- 1 azeghost azeghost 46540659 Jul  1 22:40 pokemon_training.tar.bz2
-rw-r--r-- 1 azeghost azeghost 10485760 Jul 13 21:43 pokemon_training.tar.partaa
-rw-r--r-- 1 azeghost azeghost 10485760 Jul 13 21:43 pokemon_training.tar.partab
-rw-r--r-- 1 azeghost azeghost 10485760 Jul 13 21:43 pokemon_training.tar.partac
-rw-r--r-- 1 azeghost azeghost 10485760 Jul 13 21:43 pokemon_training.tar.partad
-rw-r--r-- 1 azeghost azeghost  4597619 Jul 13 21:43 pokemon_training.tar.partae
-rw-r--r-- 1 azeghost azeghost 46540659 Jul  1 22:41 pokemon_validation.tar.bz2
-rw-r--r-- 1 azeghost azeghost 46540659 Jul  1 22:30 pokemon_validation.tar.partaa


In [17]:
# push to git
# !cd /home/azeghost/git/LMDB_Datasets
# !git add ../pokemon/*
# # !git status
# !git commit -m "pokemon_training"
# !git push orign master # Check if you have ssh token enabled for bash use otherwise enter login and password

## For validation

In [18]:
!tar -cvjf pokemon_validation.tar.bz2 /home/azeghost/git/Generative_Models/LMDB/_validation/* 

tar: Removing leading `/' from member names
/home/azeghost/git/Generative_Models/LMDB/_validation/data.mdb
/home/azeghost/git/Generative_Models/LMDB/_validation/lock.mdb


In [19]:
!split -b 200M pokemon_training.tar.bz2 "pokemon_validation.tar.part"
!ls -lh pokemon_validation.tar.part*

-rw-r--r-- 1 azeghost azeghost 45M Jul 13 21:43 pokemon_validation.tar.partaa


In [20]:
!mkdir /home/azeghost/git/LMDB_Datasets/pokemon
!mv pokemon_validation.tar.part* /home/azeghost/git/LMDB_Datasets/pokemon
!ls /home/azeghost/git/LMDB_Datasets/pokemon

mkdir: cannot create directory ‘/home/azeghost/git/LMDB_Datasets/pokemon’: File exists
home			     pokemon_training.tar.partad
pokemon_training.tar.bz2     pokemon_training.tar.partae
pokemon_training.tar.partaa  pokemon_validation.tar.bz2
pokemon_training.tar.partab  pokemon_validation.tar.partaa
pokemon_training.tar.partac


In [21]:
# !cd /home/azeghost/git/LMDB_Datasets
# !git add pokemon/*
# !git status
# !git commit -m "pokemon_validation"
# !git push orign master

# After downloading from Git combine and unzip

## combine them

In [34]:
#pokemon_training
!cat /home/azeghost/git/LMDB_Datasets/pokemon/pokemon_training.tar.part* >/home/azeghost/git/LMDB_Datasets/pokemon/pokemon_training.tar.bz2
#!cat <split files put * at the end> > <final zip name>
!ls -la /home/azeghost/git/LMDB_Datasets/pokemon

total 181820
drwxr-xr-x 2 azeghost azeghost     4096 Jul  1 22:40 .
drwxr-xr-x 6 azeghost azeghost     4096 Jul  1 22:31 ..
-rw-r--r-- 1 azeghost azeghost 46540659 Jul  1 22:40 pokemon_training.tar.bz2
-rw-r--r-- 1 azeghost azeghost 10485760 Jul  1 22:35 pokemon_training.tar.partaa
-rw-r--r-- 1 azeghost azeghost 10485760 Jul  1 22:35 pokemon_training.tar.partab
-rw-r--r-- 1 azeghost azeghost 10485760 Jul  1 22:35 pokemon_training.tar.partac
-rw-r--r-- 1 azeghost azeghost 10485760 Jul  1 22:35 pokemon_training.tar.partad
-rw-r--r-- 1 azeghost azeghost  4597619 Jul  1 22:35 pokemon_training.tar.partae
-rw-r--r-- 1 azeghost azeghost 46540659 Jul  1 22:39 pokemon_validation.tar.bz2
-rw-r--r-- 1 azeghost azeghost 46540659 Jul  1 22:30 pokemon_validation.tar.partaa


In [35]:
#pokemon_validation
!cat /home/azeghost/git/LMDB_Datasets/pokemon/pokemon_validation.tar.part* >/home/azeghost/git/LMDB_Datasets/pokemon/pokemon_validation.tar.bz2
!ls -la /home/azeghost/git/LMDB_Datasets/pokemon

total 181816
drwxr-xr-x 2 azeghost azeghost     4096 Jul  1 22:40 .
drwxr-xr-x 6 azeghost azeghost     4096 Jul  1 22:31 ..
-rw-r--r-- 1 azeghost azeghost 46540659 Jul  1 22:40 pokemon_training.tar.bz2
-rw-r--r-- 1 azeghost azeghost 10485760 Jul  1 22:35 pokemon_training.tar.partaa
-rw-r--r-- 1 azeghost azeghost 10485760 Jul  1 22:35 pokemon_training.tar.partab
-rw-r--r-- 1 azeghost azeghost 10485760 Jul  1 22:35 pokemon_training.tar.partac
-rw-r--r-- 1 azeghost azeghost 10485760 Jul  1 22:35 pokemon_training.tar.partad
-rw-r--r-- 1 azeghost azeghost  4597619 Jul  1 22:35 pokemon_training.tar.partae
-rw-r--r-- 1 azeghost azeghost 46540659 Jul  1 22:41 pokemon_validation.tar.bz2
-rw-r--r-- 1 azeghost azeghost 46540659 Jul  1 22:30 pokemon_validation.tar.partaa


## Unzip   

In [37]:
!tar -xf /home/azeghost/git/LMDB_Datasets/pokemon/pokemon_validation.tar.bz2 -C /home/azeghost/git/LMDB_Datasets
!ls -la /home/azeghost/git/LMDB_Datasets

total 40
drwxr-xr-x 7 azeghost azeghost 4096 Jul  1 22:49 .
drwxr-xr-x 5 azeghost azeghost 4096 Jun 25 08:39 ..
drwxr-xr-x 8 azeghost azeghost 4096 Jul  1 22:31 .git
drwxr-xr-x 3 azeghost azeghost 4096 Jul  1 22:49 home
drwxrwxr-x 3 azeghost azeghost 4096 Jul  1 22:32 .idea
-rw-rw-r-- 1 azeghost azeghost 8376 Jul  1 22:24 LMDB_Write_and_Read_R01.ipynb
drwxr-xr-x 3 azeghost azeghost 4096 Jul  1 22:44 pokemon
drwxr-xr-x 3 azeghost azeghost 4096 Jul  1 22:49 Pokemon


# Read the lmdb 

In [22]:
def read_many_lmdb(lmdb_dir, num_images):

    images, labels = [], []
    env = lmdb.open(lmdb_dir, readonly=True)

    # Start a new read transaction
    with env.begin() as txn:
        # Read all images in one single transaction, with one lock
        # We could split this up into multiple transactions if needed
        for image_id in range(num_images):
            data = txn.get(f"{image_id:08}".encode("ascii"))
            # Remember that it's a CIFAR_Image object 
            # that is stored as the value
            pokemon_image = pickle.loads(data)
            images.append(pokemon_image.get_image())
            labels.append(pokemon_image.label)
    env.close()
    return images, labels

In [23]:
x, y = read_many_lmdb('./LMDB/_validation', 218)

In [24]:
os.chdir('/home/azeghost/git/Generative_Models/data/')

In [25]:
import glob
import hashlib
import logging
import os
import re
import warnings

from keras.preprocessing.image import ImageDataGenerator

from utils.data_and_files.data_utils import as_bytes
from utils.reporting.logging import log_message
# from training.generators.image_iterator import LMDBImageIterator


class LMDBImageGenerator(ImageDataGenerator):
    def flow_from_lmdb_lists(self, 
                              category,
                              batch_size,
                              episode_len=None,
                              episode_shift=None,
                              color_mode='rgb',
                              class_mode=None,
                              shuffle=True,
                              seed=None,
                              save_to_dir=None,
                              save_prefix='',
                              save_format='jpg'):

        return LMDBImageIterator(self,
                             category,
                             class_mode=class_mode,
                             data_format=self.data_format,
                             batch_size=batch_size,
                             episode_len=episode_len,
                             episode_shift=episode_shift,
                             shuffle=shuffle,
                             seed=seed,
                             save_to_dir=save_to_dir,
                             save_prefix=save_prefix,
                             save_format=save_format)


def get_generators( lmdb_dir, batch_size, class_mode, episode_len=None, episode_shift=None, scaler=255.0):

    train_datagen = LMDBImageGenerator(rescale=1. / scaler)

    valid_datagen = LMDBImageGenerator(rescale=1. / scaler)

    train_generator = train_datagen.flow_from_lmdb_lists(
        category='training',
        lmdb_dir=lmdb_dir,
        batch_size=batch_size,
        class_mode=class_mode,
        episode_len=episode_len,
        episode_shift=episode_shift,
        seed=0)

    validation_generator = valid_datagen.flow_from_lmdb_lists(
        image_lists=images_list,
        category='validation',
        image_dir=image_dir,
        target_size=image_size,
        batch_size=batch_size,
        class_mode=class_mode,
        episode_len=episode_len,
        episode_shift=episode_shift,
        seed=0)

    return train_generator, validation_generator



In [26]:
import os
import numpy as np
import tensorflow as tf
from keras.preprocessing.image import Iterator, load_img, img_to_array, array_to_img
from keras import backend as K
import logging
from utils.reporting.logging import log_message
from utils.data_and_files.file_utils import get_file_path

class LMDBImageIterator(Iterator):
    """Iterator capable of reading images from a directory on disk.

    # Arguments
        image_lists: Dictionary of training images for each label.
        image_data_generator: Instance of `ImageDataGenerator`
            to use for random transformations and normalization.
        target_size: tuple of integers, dimensions to resize input images to.
        color_mode: One of `"rgb"`, `"grayscale"`. Color mode to read images.
        classes: Optional list of strings, names of sudirectories
            containing images from each class (e.g. `["dogs", "cats"]`).
            It will be computed automatically if not set.
        class_mode: Mode for yielding the targets:
            `"binary"`: binary targets (if there are only two classes),
            `"categorical"`: categorical targets,
            `"sparse"`: integer targets,
            `None`: no targets get yielded (only input images are yielded).
            `episode`: a sequence of images yielded (with time shift).
            `func`: custom transformation gets the input images and the transformed.

        batch_size: Integer, size of a batch.
        shuffle: Boolean, whether to shuffle the data between epochs.
        seed: Random seed for data shuffling.
        data_format: String, one of `channels_first`, `channels_last`.
        save_to_dir: Optional directory where to save the pictures
            being yielded, in a viewable format. This is useful
            for visualizing the random transformations being
            applied, for debugging purposes.
        save_prefix: String prefix to use for saving sample
            images (if `save_to_dir` is set).
        save_format: Format to use for saving sample images
            (if `save_to_dir` is set).
    """

    def __init__(self,
                 category,
                 lmdb_dir,
                 class_mode='categorical',
                 batch_size=32,
                 episode_len=20,
                 episode_shift=10,
                 shuffle=True,
                 seed=None,
                 save_to_dir=None,
                 save_prefix='',
                 save_format='jpeg',
                 dtype=K.floatx(),
                 number_of_records = 10
                 
                 ):
        
        self.category = category
        self.batch_size = batch_size
        
        
        self.lmdb_dir = lmdb_dir
        self.episode_len = episode_len
        self.episode_shift = episode_shift

        
        
#         self.class2id = dict(zip(classes, range(len(classes))))
#         self.id2class = dict((v, k) for k, v in self.class2id.items())
#         self.classes = np.zeros((self.samples,), dtype='int32')



        if (class_mode not in {'categorical', 'binary', 'sparse', 'episode', 'episode_flat', None}) and (not hasattr(class_mode, '__call__')):
            raise ValueError('Invalid class_mode:', class_mode,
                             '; expected one of "categorical", '
                             '"binary", "sparse", "episode", or None.')
        self.class_mode = class_mode
        self.dtype = dtype
        self.save_to_dir = save_to_dir
        self.save_prefix = save_prefix
        self.save_format = save_format

        Iterator.__init__(self, self.samples, self.batch_size, shuffle, seed)

    def _get_batches_of_transformed_samples(self, index_array):
        """For python 2.x.

        # Returns
            The next batch.
        """
        # with self.lock:
        #    index_array, current_index, current_batch_size = next(self.index_generator)
        # The transformation of images is not under thread lock
        # so it can be done in parallel

        if len(index_array) < self.batch_size:
            diff = self.batch_size//len(index_array) + 1
            index_array = np.repeat(index_array, diff, axis=0)[:self.batch_size]

#         grayscale = self.color_mode == 'grayscale'
        else:
            batch_x = np.zeros((len(index_array),) + self.image_shape, dtype=self.dtype)
            # build batch of image data
            
            env = lmdb.open(lmdb_dir, readonly=True)
            
            for i, j in enumerate(index_array):
                
                images, labels = [], []
                

                # Start a new read transaction
                with env.begin() as txn:
                    # Read all images in one single transaction, with one lock
                    # We could split this up into multiple transactions if needed
                    for image_id in index_array:
#                         data = txn.get(f"{image_id:08}".encode("ascii"))
                        data = txn.get(image_id)
    
                        pokemon_image = pickle.loads(data)
                        images.append(pokemon_image.get_image())
                        labels.append(pokemon_image.label)
                env.close()
                return images, labels
                
                x = self.image_data_generator.random_transform(x)
                x = self.image_data_generator.standardize(x)
                batch_x[i] = x

#             # optionally save augmented images to disk for debugging purposes
#             if self.save_to_dir:
#                 for i, j in enumerate(index_array):
#                     img = array_to_img(batch_x[i], self.data_format, scale=True)
#                     fname = '{prefix}_{index}_{hash}.{format}'.format(prefix=self.save_prefix,
#                                                                       index=j,
#                                                                       hash=np.random.randint(10000),
#                                                                       format=self.save_format)
#                     img.save(os.path.join(self.save_to_dir, fname))
            # build batch of labels
            if self.class_mode == 'sparse':
                batch_y = self.classes[index_array]
            elif self.class_mode == 'binary':
                batch_y = self.classes[index_array].astype(K.floatx())
            elif self.class_mode == 'categorical':
                batch_y = np.zeros((len(batch_x), self.num_class),dtype=K.floatx())
                for i, label in enumerate(self.classes[index_array]):
                    batch_y[i, label] = 1.

            elif self.class_mode is None:
                return batch_x
            else:
                return batch_x, self.class_mode(batch_x)
            return batch_x, batch_y



In [ ]:

# get keys 
#create a random batch size key list
# get records return numpy